In [ ]:
import numpy as np
from numba import cuda

In [ ]:
col = 4096; row = 4096;
def reverseArray(arr) :
    for i in range(col) :
        s = 0;
        l = row - 1;
        while (s < l) :
            arr[i][s], arr[i][l] = arr[i][l], arr[i][s];
            s += 1;
            l -= 1;

In [ ]:
dim = 4096 * 4096

threads_per_block = (32, 32)
blocks_per_grid = (128, 128)

In [ ]:
a = np.arange(dim).reshape(4096,4096).astype(np.int32)
out = np.zeros_like(a).astype(np.int32)

In [ ]:
d_arr = cuda.to_device(a)
d_out = cuda.to_device(out)

In [ ]:
%timeit reverseArray(a);

8.28 s ± 622 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from numba import jit
col = 4096; row = 4096;

@jit
def reverseArray_cpu_parallel(arr) :
    for i in range(col) :
        s = 0;
        l = row - 1;
        while (s < l) :
            arr[i][s], arr[i][l] = arr[i][l], arr[i][s];
            s += 1;
            l -= 1;

In [ ]:
%timeit reverseArray_cpu_parallel(a);

10.2 ms ± 3.33 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
@cuda.jit
def reverseArray_gpu(a, out, coalesced):
    x, y = cuda.grid(2)
    l = 4096*4096
    i=1
    ll=4096
    if coalesced == True:
        out[x][ll-y] = a[x][y]
        cuda.syncthreads()
        i+=1

    else:
        out[x][ll-y] = a[x][y]
        cuda.syncthreads()
        i+=1

In [ ]:
%timeit reverseArray_gpu[blocks_per_grid, threads_per_block](d_a,d_out,True);cuda.synchronize

1.95 ms ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
